In [1]:
import os

def get_filenames(folder_path):
    filenames = []
    for file in os.listdir(folder_path):
        if file.endswith(".jpg") or file.endswith(".tiff"):
            filenames.append(os.path.join(folder_path, file))
    return filenames

folder_path = "./DANE/przykladowe_zdj/przykladowe_zdj/"
jpg_filenames = get_filenames(folder_path)
print(jpg_filenames)

['./DANE/przykladowe_zdj/przykladowe_zdj/0a16d521-1f33-45be-bde8-376c9026fd96.tiff', './DANE/przykladowe_zdj/przykladowe_zdj/0a1d5be5-8487-4733-babb-c1c71cee876b.tiff', './DANE/przykladowe_zdj/przykladowe_zdj/0a2d8a84-07ee-4d76-a246-8625d1c2cc65.jpg', './DANE/przykladowe_zdj/przykladowe_zdj/0a3e5c11-9f35-4398-9598-da5177296461.tiff', './DANE/przykladowe_zdj/przykladowe_zdj/0a6bfb5d-a4d7-4e4c-a1ba-22f259078b3d.tiff', './DANE/przykladowe_zdj/przykladowe_zdj/0a7cd32d-6127-433c-b122-a6eb30a53f9e.tiff', './DANE/przykladowe_zdj/przykladowe_zdj/0ad15aa1-78bf-4e4f-86c4-f0c395785e38.tiff', './DANE/przykladowe_zdj/przykladowe_zdj/0b645a67-6f38-4da2-95e2-7460f800c275.tiff', './DANE/przykladowe_zdj/przykladowe_zdj/0b7397d3-6cbf-4616-b5eb-44022be04a63.tiff']


In [2]:
import cv2
import pytesseract
from pytesseract import Output
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

def detect_image_table(filepaths):
    path_to_tesseract = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    pytesseract.pytesseract.tesseract_cmd = path_to_tesseract  # Change this path to your Tesseract executable
    model = MobileNetV2(weights='imagenet', include_top=False)

    def is_image(image):
        image = cv2.resize(image, (224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)
        image = tf.expand_dims(image, axis=0)

        features = model.predict(image)
        has_image = features.sum() > 0
        return has_image

    def is_table(image):
        d = pytesseract.image_to_data(image, output_type=Output.DICT)
        has_table = False
        for i in range(len(d['level'])):
            if d['level'][i] == 2:  # Level 2 corresponds to block-level elements
                print(d['block_num'][i])
                if d['block_num'][i] != 0:
                    if d['block_num'][i] != prev_block_num:
                        if d['conf'][i] < 50:
                            has_table = True
            prev_block_num = d['block_num'][i]
        return has_table

    data = []
    for filepath in filepaths:
        img = cv2.imread(filepath)
        has_table = is_table(img)
        has_image = is_image(img)
        data.append([filepath, has_image, has_table])

    df = pd.DataFrame(data, columns=['filepath', 'hasImage', 'hasTable'])
    return df


filepaths = jpg_filenames
result_df = detect_image_table(filepaths)
print(result_df)

ModuleNotFoundError: No module named 'tensorflow'